# Mexico City Metro Map Tutorial

This notebook guides you through creating an interactive map of Mexico City's metro system using Python, GeoPandas, and Folium.

## Setup

First, let's install the required packages:

In [ ]:
!pip install geopandas folium requests

Now, let's import the necessary libraries:

In [ ]:
import geopandas as gpd
import folium
import json
import requests
import time

## Step 1: Convert Shapefile to GeoJSON

First, we need to upload the shapefile. In Google Colab, you can use the file upload widget:

In [ ]:
from google.colab import files
uploaded = files.upload()

# Assuming the uploaded file is named 'STC_Metro_estaciones_utm14n.shp'
input_shp = 'STC_Metro_estaciones_utm14n.shp'

Now, let's convert the shapefile to GeoJSON:

In [ ]:
def convert_shp_to_geojson(input_shp, output_geojson):
    gdf = gpd.read_file(input_shp)
    gdf.to_file(output_geojson, driver='GeoJSON')
    print(f"Conversion complete. GeoJSON file saved as {output_geojson}")
    return gdf

output_geojson = 'mexico_city_subway_stations.geojson'
gdf = convert_shp_to_geojson(input_shp, output_geojson)

## Step 2: Create the Interactive Map

Now, let's create the interactive map with stations and isochrones:

In [ ]:
# OpenRouteService API key
ORS_API_KEY = 'YOUR_API_KEY_HERE'

def get_isochrone(lat, lng, time_range):
    url = "https://api.openrouteservice.org/v2/isochrones/foot-walking"
    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': ORS_API_KEY,
        'Content-Type': 'application/json; charset=utf-8'
    }
    data = {
        "locations": [[lng, lat]],
        "range": [time_range * 60],  # Convert minutes to seconds
        "attributes": ["area"],
        "units": "km"
    }
    
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching isochrone: {response.status_code}, {response.text}")
        return None

def get_station_color(station_type):
    color_map = {
        'Terminal / Transbordo': 'orange',
        'Terminal': 'red',
        'Transbordo': 'blue',
        'Intermedia': 'green'
    }
    return color_map.get(station_type, 'gray')

def create_map():
    # Load GeoJSON file
    with open('mexico_city_subway_stations.geojson', 'r') as f:
        metro_stations = json.load(f)

    # Create a map centered on Mexico City
    m = folium.Map(location=[19.4326, -99.1332], zoom_start=11)

    # Add stations and isochrones to the map
    for feature in metro_stations['features']:
        properties = feature['properties']
        coordinates = feature['geometry']['coordinates']
        lng, lat = coordinates[0], coordinates[1]
        
        # Add station marker
        folium.CircleMarker(
            location=[lat, lng],
            radius=8,
            popup=folium.Popup(f"<b>{properties['NOMBRE']}</b><br>Line: {properties['LINEA']}<br>Type: {properties['TIPO']}<br>Borough: {properties['ALCALDIAS']}<br>Year: {properties['AÑO']}", max_width=300),
            color='black',
            fill=True,
            fillColor=get_station_color(properties['TIPO']),
            fillOpacity=0.7
        ).add_to(m)
        
        # Get and add isochrone
        isochrone = get_isochrone(lat, lng, 15)  # 15-minute isochrone
        if isochrone:
            folium.GeoJson(
                isochrone,
                style_function=lambda x: {'fillColor': 'green', 'color': 'green', 'weight': 2, 'fillOpacity': 0.3}
            ).add_to(m)
        
        time.sleep(1)  # Sleep to avoid hitting API rate limits

    return m

# Create and display the map
m = create_map()
m

## Save the Map

To save the map as an HTML file that you can download:

In [ ]:
m.save("mexico_city_metro_map.html")
files.download("mexico_city_metro_map.html")